## Forest Health Threshold Update - Results

### Setup

In [2]:
# import packages
import arcpy
import pandas as pd
from pathlib import Path
from utils import *
import plotly.express as px
import plotly.graph_objects as go
from arcgis import GeoAccessor, GeoSeriesAccessor
import pickle

# setup workspace variables
arcpy.env.workspace = r"F:\\GIS\\PROJECTS\\ForestHealth_Intiative\\ThresholdUpdate\\Data\\ForestHealth_ThresholdUpdate.gdb"
arcpy.env.overwriteOutput = True

#format paths
sdeBase   = Path("F:\GIS\DB_CONNECT\Vector.sde")
# workspace = Path("F:\GIS\PROJECTS\ForestHealth_Intiative\ThresholdUpdate\Data\ForestHealth_ThresholdUpdate.gdb")
downloads = Path("F:\GIS\PROJECTS\ForestHealth_Intiative\ThresholdUpdate\Data\Download\SNV_RRK")
output    = Path("F:\GIS\PROJECTS\ForestHealth_Intiative\ThresholdUpdate\AnalysisProduct")

# set style variables
template = 'plotly_white'
font     = 'Calibri'
# colors for tables
headerColor = '#6680a8'
rowColor = 'white'
lastrowColor = '#eeeeee'

In [ ]:
# list datasets in workspace
feature_data = arcpy.ListFeatureClasses()
raster_data  = arcpy.ListRasters()
table_data   = arcpy.ListTables()
# print feature classes, then rasters, then tables
print("Feature Classes:")
for fc in feature_data:
    print(fc)
print("\nRasters:")
for raster in raster_data:
    print(raster)
print("\nTables:")
for table in table_data:
    print(table)

## Stand Density

In [3]:
# 
raster_path = "stand_density_threshold_assessment"

# Describe the raster and get the attribute table
fields = [f.name for f in arcpy.ListFields(raster_path)]
rows = arcpy.da.SearchCursor(raster_path, fields)

# Build DataFrame
data = [list(row) for row in rows]
df = pd.DataFrame(data, columns=fields)
df

,OBJECTID,Value,Count,Acres
0,1,0,287827.0,64011.285665
1,2,1,340587.0,75744.845865


In [4]:
raster_path = "stand_density_threshold_assessment"
# Describe the raster and get the attribute table
fields = [f.name for f in arcpy.ListFields(raster_path)]
rows = arcpy.da.SearchCursor(raster_path, fields)

# Build DataFrame
data = [list(row) for row in rows]
df = pd.DataFrame(data, columns=fields)


# define the target status based on the raster value
df["TargetStatus"] = df["Value"].map({
    1 : "Below Threshold",
    0 : "Above Threshold",
    -1: "N/A",
})
#group by target status and acres
df_grouped = df.groupby("TargetStatus")["Acres"].sum().reset_index()
# calculate percentage of acres in each target status
df_grouped["Percentage"] = (df_grouped["Acres"] / df_grouped["Acres"].sum()) * 100

# format the percentage column
df_grouped["Percentage"] = df_grouped["Percentage"].round(1)

# get copy for chart
df = df_grouped.copy()
# bar chart of target status by acres
fig = px.bar(df, x="TargetStatus", y="Acres", 
            text=df['Percentage'].astype(str) + '%',
            title="Stand Density",
            color_discrete_sequence=["#ABCD66", "#728944"],
            custom_data=["TargetStatus"]
            )
fig.update_traces(marker_line_color='rgb(8,48,107)',
                    marker_line_width=0.1, opacity=0.9,
                    textfont_size=14, textposition="inside",
                    hovertemplate="<br>".join(
                        ["<b>%{y:,.0f} acres</b> of the forested area are", 
                            "<i>%{customdata[0]}</i> for resilience"]
                    )
                    + "<extra></extra>",
            )
fig.update_layout(title_x=0.5, title_font_size=20,
                    uniformtext_minsize=8, uniformtext_mode='hide',
                    title_font_color='#6680a8',
                    xaxis_title="Stand Density - SNRRK",
                    yaxis_title="Acres",
                    hovermode="x unified",
                    template=template,
                    legend=dict(title_text='',
                                orientation="h",
                                yanchor="bottom",
                                y=1.02,
                                xanchor="right",
                                x=1),
                    legend_title_text='',
                    font_family=font,
                    font_color='#6680a8',
                    plot_bgcolor='white',
                    showlegend=True
                        )
# no legendd
fig.update_traces(showlegend=False)
fig.show()
# save to HTML
fig.write_html(config = {"displayModeBar": False},
               file="DataVisualizations/StandDensity_Chart_SNRRK.html", 
               full_html= False, 
               include_plotlyjs='cdn')

## Seral Stage

In [10]:
# stacked bar chart of seral stage by canopy cover class
df = pd.read_csv(output / "seral_stage_with_classification.csv")
df['Seral Stage'].unique()

array(['Early', 'Mid (closed canopy)', 'Mid (open canopy)',
       'Late (open canopy)', 'Late (closed canopy)'], dtype=object)

In [13]:
df

,Forest Type,Category,Desired % Range,Classification,Current Area %,Area (m²),Acres,SeralStage,Percentage
0,Jeffrey Pine,Early,5–15%,below target,2.12,3492900.0,863.0,Early Seral,0.6
1,Jeffrey Pine,Mid (closed canopy),5–10%,above target,28.48,46978200.0,11609.0,Mid Seral,8.3
2,Jeffrey Pine,Mid (open canopy),25–30%,on target,27.75,45772200.0,11311.0,Mid Seral,8.1
3,Jeffrey Pine,Late (open canopy),45–50%,below target,27.28,44996400.0,11119.0,Late Seral,8.0
4,Jeffrey Pine,Late (closed canopy),5–10%,above target,14.37,23700600.0,5857.0,Late Seral,4.2
5,White Fir,Early,10–20%,below target,3.45,8957700.0,2213.0,Early Seral,1.6
6,White Fir,Mid (closed canopy),5–15%,above target,37.27,96792300.0,23918.0,Mid Seral,17.1
7,White Fir,Mid (open canopy),10–15%,above target,22.73,59042700.0,14590.0,Mid Seral,10.4
8,White Fir,Late (open canopy),30–40%,below target,19.41,50410800.0,12457.0,Late Seral,8.9
9,White Fir,Late (closed canopy),20–30%,below target,17.14,44503200.0,10997.0,Late Seral,7.9


In [24]:
# stacked bar chart of seral stage by canopy cover class
df = pd.read_csv(output / "seral_stage_with_classification.csv")

# rename columns for clarity
df.rename(columns = {
    'Seral Stage':'Category',
    'Area (acres)':'Acres'
    }, inplace = True)

# classify seral stage by category Early Seral Opne - Early Seral, etch
df['SeralStage'] = df['Category'].replace(
    {'Early': 'Early Seral',
    'Mid (closed canopy)': 'Mid Seral',
    'Mid (open canopy)': 'Mid Seral',
    'Late (open canopy)': 'Late Seral',
    'Late (closed canopy)': 'Late Seral'
    }
)
# melt then group by seral stage and category and sum acres
df = df.melt(id_vars=['SeralStage', 'Category'], value_vars=['Acres'])
df = df.groupby(['SeralStage', 'Category']).sum().reset_index()
# remove the "SeralStage" column from the melted DataFrame
df = df.drop(columns=['variable'])
# change value to Acres
df.rename(columns = {
    'value':'Acres'
    }, inplace = True)

# order the categories by the order of the seral stage
df['SeralStage'] = pd.Categorical(df['SeralStage'],
                        categories=['Early Seral', 'Mid Seral', 'Late Seral'],
                        ordered=True)

## get the percentage of acres in each category
df['Percentage'] = ((df['Acres'] / df['Acres'].sum()) * 100).round(1)

fig = px.bar(df, x="SeralStage", y="Acres", 
            color="Category",
            text=df['Percentage'].astype(str) + '%',
            title="Seral Stage by Canopy Cover Class",
            color_discrete_sequence=["#ABCD66", "#728944"],
            custom_data=["Category"],
            category_orders={"SeralStage": ['Early Seral', 'Mid Seral', 'Late Seral']},
            )
fig.update_traces(marker_line_color='rgb(8,48,107)',
                    marker_line_width=0.1, opacity=0.9,
                    textfont_size=14, textposition="inside",
                    hovertemplate="<br>".join(
                        ["<b>%{y:,.0f}</b> acres of the forested area is", 
                            "in class <i>%{customdata[0]}</i>"]
                    )
                    + "<extra></extra>",
            )
fig.update_layout(title_x=0.5, title_font_size=20,
                    uniformtext_minsize=8, uniformtext_mode='hide',
                    title_font_color='#6680a8',
                    xaxis_title="Seral Stage",
                    yaxis_title="Acres",
                    hovermode="x unified",
                    template=template,
                    legend=dict(title_text='',
                                orientation="h",
                                yanchor="bottom",
                                y=1.02,
                                xanchor="right",
                                x=1),
                    legend_title_text='',
                    font_family=font,
                    font_color='#6680a8',
                    plot_bgcolor='white',
                    showlegend=True
                        )
# no legendd
fig.update_traces(showlegend=False)
fig.show()
# save to HTML
fig.write_html(config = {"displayModeBar": False},
               file="DataVisualizations/CompositionAge_Chart.html", 
               full_html= False, 
               include_plotlyjs='cdn')

In [16]:
df

,SeralStage,Category,variable,value
0,Early Seral,Early,Acres,863.0
1,Mid Seral,Mid (closed canopy),Acres,11609.0
2,Mid Seral,Mid (open canopy),Acres,11311.0
3,Late Seral,Late (open canopy),Acres,11119.0
4,Late Seral,Late (closed canopy),Acres,5857.0
5,Early Seral,Early,Acres,2213.0
6,Mid Seral,Mid (closed canopy),Acres,23918.0
7,Mid Seral,Mid (open canopy),Acres,14590.0
8,Late Seral,Late (open canopy),Acres,12457.0
9,Late Seral,Late (closed canopy),Acres,10997.0


## Functional Fire

In [ ]:
# get data frame from feature class
df_fire = pd.DataFrame.spatial.from_featureclass("ID_MGMTzones_FunctionalFire_DominantClass_Tahoe_SNVRRK")

df_fire['Name'] = df_fire['Name'].replace(
    to_replace=[
        "Desolation Wilderness",
        "Granite Chief Wilderness",
        "Mt. Rose Wilderness"
    ],
    value="Wilderness"
)

lookup_dict = {
    1: "Low Severity",
    2: "Moderate Severity",
    3: "High Severity"
}
dfMgmt_Fire = df_fire.groupby(['Name', 'gridcode']).agg({'Acres': 'sum'}).reset_index()
dfMgmt_Fire.rename(columns={"gridcode": "Fire Severity", "Acres": "Acres"}, inplace=True)
dfMgmt_Fire['Fire Severity'] = dfMgmt_Fire['Fire Severity'].map(lookup_dict)
dfMgmt_Fire['Percentage'] = ((dfMgmt_Fire['Acres'] / dfMgmt_Fire['Acres'].sum()) * 100).round(1)
dfMgmt_Fire['Acres'] = dfMgmt_Fire['Acres'].round(1)
# filter out Name ''
dfMgmt_Fire = dfMgmt_Fire[dfMgmt_Fire['Name'] != '']
dfMgmt_Fire.to_csv(output / "FireSeverity_ManagementZones.csv", index=False)

In [ ]:
# stacked bar chart of severity by management zone
fig = px.bar(dfMgmt_Fire, x="Name", y="Acres", 
            color="Fire Severity",
            text=dfMgmt_Fire['Percentage'].astype(str) + '%',
            title="Fire Severity Probability by Management Zone",
            labels={"Acres": "Acres", "Name": "Management Zone"},
            color_discrete_sequence=["#91bfdb", "#ffffbf", "#fc8d59"],
            custom_data=["Fire Severity"]
            )
fig.update_traces(marker_line_color='rgb(8,48,107)',
                    marker_line_width=0.1, opacity=0.9,
                    textfont_size=14, textposition="inside",
                    hovertemplate="<br>".join(
                        ["<b>%{y:,.0f}</b> of the forested area is", 
                            "likely to burn at <i>%{customdata[0]}</i>"]
                    )
                    + "<extra></extra>",
            )
fig.update_layout(title_x=0.5, title_font_size=20,
                    uniformtext_minsize=8, uniformtext_mode='hide',
                    title_font_color='#6680a8',
                    xaxis_title="Management Zone",
                    yaxis_title="Acres",
                    hovermode="x unified",
                    template=template,
                    legend=dict(title_text='',
                                orientation="h",
                                yanchor="bottom",
                                y=1.02,
                                xanchor="right",
                                x=1),
                    legend_title_text='Fire Severity',
                    font_family=font,
                    font_color='#6680a8',
                    plot_bgcolor='white',
                    showlegend=True
                        )
fig.show()
# save to HTML
fig.write_html(config = {"displayModeBar": False},
               file="DataVisualizations/FireSeverity_ManagementZone.html", 
               full_html= False, 
               include_plotlyjs='cdn')

## Performance Meassures

In [ ]:
# Fuels Treatment Chart
df = get_data_forest_fuel()
plot_forest_fuel(df)